In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-pdf/Joseph Babcock Raghav Bali - Generative AI with Python and TensorFlow 2_ Create images text and music with VAEs GANs LSTMs Transformer models-Packt Publishing (2021).pdf
/kaggle/input/ai-pdf/Ben Auffarth - Generative AI with LangChain_ Build large language model (LLM) apps with Python ChatGPT and other LLMs-Packt (2023).pdf
/kaggle/input/ai-pdf/Chris Fregly Antje Barth Shelbee Eigenbrode - Generative AI on AWS_ Building Context-Aware Multimodal Reasoning Applications-OReilly Media Inc. (2023).pdf
/kaggle/input/ai-pdf/Python for Programmers with Big Data and Artificial Intelligence Case Studies by Paul J. Deitel Harvey Deitel (z-lib.org).pdf


In [79]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_HF = user_secrets.get_secret("Huggingface")
secret_value_OA = user_secrets.get_secret("OPEN_API_KEY")
secret_value_PC = user_secrets.get_secret("PINECONE")
secret_value_Wbd = user_secrets.get_secret("wandb login")
secret_value_A_KEY = user_secrets.get_secret("ASTRA_API_KEY")
secret_value_A_DB = user_secrets.get_secret("ASTRA_DB_ID")
secret_value_P_ENV = user_secrets.get_secret("PINECONE_ENV")

In [80]:
#!pip install langchain

In [81]:
from langchain.document_loaders import PyPDFLoader
loaders=[
    #PyPDFLoader('/kaggle/input/ai-pdf/Ben Auffarth - Generative AI with LangChain_ Build large language model (LLM) apps with Python ChatGPT and other LLMs-Packt (2023).pdf'),
    #PyPDFLoader('/kaggle/input/ai-pdf/Chris Fregly Antje Barth Shelbee Eigenbrode - Generative AI on AWS_ Building Context-Aware Multimodal Reasoning Applications-OReilly Media Inc. (2023).pdf'),
    #PyPDFLoader('/kaggle/input/ai-pdf/Joseph Babcock Raghav Bali - Generative AI with Python and TensorFlow 2_ Create images text and music with VAEs GANs LSTMs Transformer models-Packt Publishing (2021).pdf'),
    #PyPDFLoader('/kaggle/input/ai-pdf/Python for Programmers with Big Data and Artificial Intelligence Case Studies by Paul J. Deitel Harvey Deitel (z-lib.org).pdf')
    PyPDFLoader('/kaggle/input/icea-report/ICEA-IDC-Re-commerce-Report.pdf')
]

docs=[]
for loader in loaders:
    docs.extend(loader.load())

In [82]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=300
)

splits=text_splitter.split_documents(docs)

In [83]:
#pip install openai

In [84]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings(openai_api_key=secret_value_OA)

In [85]:
#!pip install FAISS-cpu

In [86]:
#pip install tiktoken

In [87]:
from langchain.vectorstores import FAISS

vectordb=FAISS.from_documents(splits,embeddings)
vectordb.save_local('RAG_BOT_')

In [88]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(openai_api_key=secret_value_OA,model_name="gpt-3.5-turbo",temperature=0)

In [89]:
from langchain.prompts import PromptTemplate
template="""Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question:{question}
Helpful Answer:"""
QA_CHAIN_PROMPT=PromptTemplate(input_variables=["context","Question"],template=template)

#run chain
from langchain.chains import RetrievalQA
qa_chain=RetrievalQA.from_chain_type(llm,
                                     retriever=vectordb.as_retriever(),
                                     return_source_documents=True,
                                     chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})#search_type="mmr"

In [90]:
from langchain.memory import ConversationBufferMemory
memory=ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [91]:
""""from langchain.chains.conversation.memory import ConversationBufferWindowMemory
memory=ConversationBufferWindowMemory(k=1,
                               memory_key="chat_history",
                               return_messages=True)
                               """

'"from langchain.chains.conversation.memory import ConversationBufferWindowMemory\nmemory=ConversationBufferWindowMemory(k=1,\n                               memory_key="chat_history",\n                               return_messages=True)\n                               '

In [92]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa=ConversationalRetrievalChain.from_llm(
    llm,
    verbose=True,
    retriever=retriever,
    memory=memory)#verbose=True

In [93]:
q1="LEADING SECONDHAND SMARTPHONE BRANDS? "
result=qa({"question":q1})
result["answer"]



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
shipments, secondhand vendor play is distinct. 
Though the market leader, Xiaomi, holds the pole 
position in the secondhand market as well, there 
are variations as we move down the grid. Worth 
mentioning is Apple which has a 
disproportionately high share in the secondhand 
market when compared to new smartphone 
shipments, especially within the organized 
segment. The new iPhones with a higher ASP of 
US$700+ are out of reach for the mass consumer 
segment, however owing to the aspirational 
value attached, command a much higher share 
in the secondhand devices market. Similarly, 
OnePlus which holds a share of 2% in the new 
smartphone market, features in the top 5 vendor 
list in the organized 

'The leading secondhand smartphone brands mentioned in the provided context are Xiaomi, Apple, and OnePlus. Xiaomi is the market leader in the secondhand market, Apple has a disproportionately high share in the secondhand market compared to new smartphone shipments, and OnePlus features in the top 5 vendor list in the organized segment for secondhand devices.'

In [94]:
q2="what are top 3 brands"
result=qa({"question":q2})
result["answer"]



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: LEADING SECONDHAND SMARTPHONE BRANDS? 
Assistant: The leading secondhand smartphone brands mentioned in the provided context are Xiaomi, Apple, and OnePlus. Xiaomi is the market leader in the secondhand market, Apple has a disproportionately high share in the secondhand market compared to new smartphone shipments, and OnePlus features in the top 5 vendor list in the organized segment for secondhand devices.
Follow Up Input: what are top 3 brands
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
-------

'The leading secondhand smartphone brands mentioned in the provided context are Xiaomi, Apple, and OnePlus.'